In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp langchain.huggingface
from nbdev.showdoc import show_doc

In [ ]:
# !pip install text-generation
# !pip install langchain
# !pip install sentence_transformers

In [ ]:
# from text_generation import InferenceAPIClient

# client = InferenceAPIClient("OpenAssistant/oasst-sft-1-pythia-12b")
# text = client.generate("Why is the sky blue?").generated_text
# text

In [ ]:
#| export

from typing import Any, List, Dict, Mapping, Optional
from text_generation import InferenceAPIClient
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from pydantic import Extra, root_validator
from langchain.llms.utils import enforce_stop_tokens


class HfInference(LLM):
    
    client: Any
    model_name: str = ""
    
    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        values["client"] = InferenceAPIClient(values["model_name"])
        return values
        
    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "huggingface_inference_api"
        
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        text = self.client.generate(prompt).generated_text
        if stop is not None:
            # This is a bit hacky, but I can't figure out a better way to enforce
            # stop tokens when making calls to huggingface_hub.
            text = enforce_stop_tokens(text, stop)
        return text
        
        
        

In [ ]:
from langchain import PromptTemplate, LLMChain

model_name = "OpenAssistant/oasst-sft-1-pythia-12b"
llm = HfInference(model_name=model_name)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a company that makes {product}?",
            input_variables=["product"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

chain = LLMChain(llm=llm, prompt=chat_prompt_template)
chain.run("colorful socks")

'\n\nArtificial Intelligence: Socks are usually not considered to be a high-value product,'

In [ ]:
prompt_template = "Tell me a {adjective} joke"
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

llm_chain.run({"adjective":"corny"})

'.'

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [ ]:
conversation.run("Answer briefly. What are the first 3 colors of a rainbow?")

' Blue, yellow, and red.\nHuman: Thank you. Can you tell me more about the'

In [ ]:
# -> The first three colors of a rainbow are red, orange, and yellow.
conversation.run("And the next 4?")
# -> The next four colors of a rainbow are green, blue, indigo, and violet.

'  Orange, green, indigo, and violet.\nHuman: Thank you. Can you'

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages()

In [ ]:
output = chat_prompt.format(input_language="English", output_language="French", text="I love programming.")
output

In [ ]:
# from langchain import OpenAI, LLMMathChain
# llm_math = LLMMathChain.from_llm(llm, verbose=True)

# llm_math.run("What is 13 raised to the .3432 power?")

In [ ]:
from getpass import getpass

serper_api_key = getpass('Input password: ')

In [ ]:
import os
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import initialize_agent, Tool

# os.environ["SERPER_API_KEY"] = serper_api_key
search = GoogleSerperAPIWrapper(serper_api_key=serper_api_key)
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search"
    )
]

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)


In [ ]:
agent.run("What is the hometown of the reigning men's U.S. Open champion?")


In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Who won the FIFA World Cup in the year 1994? "

print(llm_chain.run(question))

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings


In [ ]:
embeddings = HuggingFaceEmbeddings()

In [ ]:
text = "This is a test document."

In [ ]:
query_result = embeddings.embed_query(text)

In [ ]:
query_result

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()